In [5]:
using PyCall
cma = pyimport("cma")

PyObject <module 'cma' from '/home/amjalled/.local/lib/python3.8/site-packages/cma/__init__.py'>

In [7]:
using DrWatson
@quickactivate :Catalyst

In [8]:
using Distributed
N = 10
addprocs(N)

10-element Array{Int64,1}:
 12
 13
 14
 15
 16
 17
 18
 19
 20
 21

In [9]:
@everywhere using DrWatson
@everywhere @quickactivate :Catalyst

      From worker 14:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 21:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 15:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 12:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 19:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 17:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 18:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 20:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 13:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 16:	 Activating environment at `~/Repos/advection-diffusion-cata

In [10]:
function calibrate(cma,N)
    Init = [0.013235189689493238, 1., 1., 1., 1.] #initial guess
    opts = cma.CMAOptions()
    opts["bounds"] = [[0, 0, 0, 0, 0], [1, 50, 50, 30, 30]] 
    opts["popsize"] = N
    weights = cma.recombination_weights.RecombinationWeights(N)
    opts["CMA_recombination_weights"] = weights
    es = cma.CMAEvolutionStrategy(Init, 0.5, opts) 
    
    input_exp = []
    output_exp = []
    for row in CSV.File(datadir("experiment/SBA15_interp_12.csv"); delim = ",")
        push!(input_exp, row.I)
        push!(output_exp, row.O)
    end
    n = 2
    nels = (5n, convert(Int,2.5n),n)
    S = Vec((0.0,0.0,0.0))
    E = Vec((1.0,0.5,0.2))
    grid = generate_grid(Tetrahedron, nels, S, E)
    faces = union(getfaceset(grid,"top"),getfaceset(grid,"bottom"),getfaceset(grid,"left"),getfaceset(grid,"right"),
                   getfaceset(grid,"front"),getfaceset(grid,"back"))
    addfaceset!(grid, "1", faces)
    
    while isempty(es.stop()) 
        solutions = es.ask() 
        fitness = pmap(function eval(input_args) 
                            try 
                               return Catalyst.solve(input_args[1], input_args[2], input_args[3], 
                                          input_exp, output_exp, progress=false, 
                                          microcomp_type=:nonlinear, 
                                          Q=input_args[4], kₙ=input_args[5], 
                                          calibration=true,w=(2.2954e-4*(1/0.37)),micromesh=grid)
                             catch err #in case we loose newton convergence in a microscopic computation 
                                return 1e10
                             end
                        end
                            ,solutions) 
        es.tell(solutions,fitness) 
        es.logger.add() 
        es.disp() 
    end
end

calibrate (generic function with 1 method)

In [ ]:
@time calibrate(cma, N)